In [1]:
import pandas as pd
import numpy as np
from datetime import date,timedelta
import warnings
from gnepias import transparency as gntp
from gnepias import epys as gnepys
warnings.filterwarnings("ignore")

configure = pd.read_json("configure.json")

In [166]:
gip_history_call = gntp.transparency_call(
    method = "POST",
    service = "electricity-service",
    endpoint = "/v1/markets/idm/data/transaction-history",
    body = {
        "startDate" : configure["settings"]["start_date"],
        "endDate" : configure["settings"]["end_date"],
    },
    response_type = "dataframe",
)

#rename columns
gip_history_call = gip_history_call.rename(columns={
    "date": "İşlem Zamanı",
    "price": "İşlem Fiyatı",
    "quantity": "Hacim",
    "contractName": "Kontrat",
})

gip_history_call = gip_history_call.drop(columns=["id","hour"])

In [167]:
gip_history_call

,İşlem Zamanı,Kontrat,İşlem Fiyatı,Hacim
0,2024-06-05T00:00:01+03:00,PH24060508,2360.3,2
1,2024-06-05T00:00:09+03:00,PH24060508,2360.8,2
2,2024-06-05T00:00:23+03:00,PH24060507,1898.7,20
3,2024-06-05T00:00:31+03:00,PH24060505,2199.9,10
4,2024-06-05T00:00:32+03:00,PH24060505,2200.0,24
...,...,...,...,...
35993,2024-06-06T23:59:54+03:00,PH24060708,2068.1,8
35994,2024-06-06T23:59:54+03:00,PH24060712,1901.5,123
35995,2024-06-06T23:59:55+03:00,PH24060708,2068.1,7
35996,2024-06-06T23:59:57+03:00,PH24060708,2068.0,8


In [3]:
#year of start date
start_date = configure["settings"]["start_date"]
end_date = configure["settings"]["end_date"]

start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

day_year = end_date.strftime("%y")
day_month = end_date.strftime("%m")
day_day = end_date.strftime("%d")
day_contract_filter = "PH" + day_year + day_month + day_day

In [4]:
list_of_contracts = gip_history_call["contractName"].unique()

target_day_contracts = gip_history_call[gip_history_call["contractName"].str.contains(day_contract_filter)]["contractName"].unique()
target_day_contracts = sorted(target_day_contracts)

In [12]:
#filtering the data by the contract
target_contract_data = gip_history_call[gip_history_call["contractName"] == target_day_contracts[0]]
target_contract_data["date"] = pd.to_datetime(target_contract_data["date"])

In [79]:
def time_filter(first_n_mins,last_n_mins,target_contract_data,filter_type):
    contract_opening_time = target_contract_data["date"].min()
    contract_closing_time = target_contract_data["date"].max()

    contract_opening_time = contract_opening_time.replace(minute=0, second=0, microsecond=0)
    contract_closing_time = contract_closing_time.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)

    if filter_type == "İlk-İlk":
        filter_opening_time = contract_opening_time + timedelta(minutes=first_n_mins)
        filter_closing_time = contract_opening_time + timedelta(minutes=last_n_mins)
    elif filter_type == "İlk-Son":
        filter_opening_time = contract_opening_time + timedelta(minutes=first_n_mins)
        filter_closing_time = contract_closing_time - timedelta(minutes=last_n_mins)
    elif filter_type == "Son-Son":
        filter_opening_time = contract_closing_time - timedelta(minutes=last_n_mins)
        filter_closing_time = contract_closing_time - timedelta(minutes=first_n_mins)
    return target_contract_data[(target_contract_data["date"] >= filter_opening_time) & (target_contract_data["date"] <= filter_closing_time)]

In [144]:
def quantity_filter(first_n_quantities,last_n_quantities,target_contract_data,filter_type):
    target_contract_data["cumulative_quantitiy"] = target_contract_data["quantity"].cumsum()
    if filter_type == "İlk-İlk":
        filter_opening_quantity = first_n_quantities
        filter_closing_quantity = last_n_quantities
    elif filter_type == "İlk-Son":
        filter_opening_quantity = first_n_quantities
        filter_closing_quantity = target_contract_data["cumulative_quantitiy"].max() - last_n_quantities
    elif filter_type == "Son-Son":
        filter_opening_quantity = target_contract_data["cumulative_quantitiy"].max() - last_n_quantities
        filter_closing_quantity = target_contract_data["cumulative_quantitiy"].max() - first_n_quantities

    target_contract_data = target_contract_data[(target_contract_data["cumulative_quantitiy"] >= filter_opening_quantity) & (target_contract_data["cumulative_quantitiy"] <= filter_closing_quantity)]
    
    return target_contract_data[(target_contract_data["cumulative_quantitiy"] >= filter_opening_quantity) & (target_contract_data["cumulative_quantitiy"] <= filter_closing_quantity)]


In [160]:
a = quantity_filter(0,120,target_contract_data,"İlk-İlk")

In [161]:
a

,date,hour,contractName,price,quantity,id,cumulative_quantitiy
12389,2024-06-05 18:00:10+03:00,18:00,PH24060600,2672.0,1,3686941707,1
12392,2024-06-05 18:00:10+03:00,18:00,PH24060600,2672.0,1,3686941769,2
12395,2024-06-05 18:00:11+03:00,18:00,PH24060600,2671.9,1,3686941752,3
12398,2024-06-05 18:00:13+03:00,18:00,PH24060600,2671.9,1,3686941709,4
12403,2024-06-05 18:00:14+03:00,18:00,PH24060600,2671.9,1,3686941714,5
12409,2024-06-05 18:00:15+03:00,18:00,PH24060600,2671.9,1,3686941816,6
12412,2024-06-05 18:00:16+03:00,18:00,PH24060600,2671.9,1,3686941817,7
12415,2024-06-05 18:00:17+03:00,18:00,PH24060600,2671.9,1,3686941820,8
12438,2024-06-05 18:01:00+03:00,18:01,PH24060600,2649.9,100,3686941782,108


In [146]:
a["quantity"].sum()

15545